# Project setup

#### Create the project directory
- mkdir drf
- cd drf

#### Create a virtual environment to isolate our package dependencies locally
- python3 -m venv env
- . myvenv/bin/activate  # On Windows use `myenv\Scripts\activate`

#### Install Django and Django REST framework into the virtual environment
- pip install django
- pip install djangorestframework
- pip install pygments  #for code highlighting

`pip freeze requirements` to check if all requirements installed successfully (create and add requriements into requirements file).

#### Set up a new project with a single application

- django-admin startproject tutorial .  

`Note the trailing '.' character `to tell django start project in current directory`

#### Create first app

- django-admin startapp snippets

Go to `tutorial/settings.py` and add `rest_framework` and `snippets.apps.SnippetsConfig` in installed apps to register the apps with tutorial project

#### Now sync your database for the first time:

- python manage.py migrate

#### Create Super User

- python manage.py runserver

We are ready to roll

***

## [Serializer](https://www.youtube.com/watch?v=MsRvqMxOryM)

Serializer packages and unpackages the data,

> Packages when data is going to server, Called De-serialize.

> Unpackages when data is coming from server (Usually Tabular data into json or other formats), Called Serialization.

Using a serializer, we can specify what fields should be present in the JSON representation of the model.


### pickling/serialization/Marshalling/Flattening

- converting a python object hierarchy into a byte stream
- keeps track of objects already serialized
- can be used to save and restore class instances
- json is a text serialization format, while pickle is a binary serialization format, not human readable.
- json is univarsal, pickle is python specific.

1. dump - Writes a pickled representation of object obj to file. # pickle.dump(x,f), x=7
2. dumps - Returns the pickled representation of obj as bytes object. Does not write to file # pickle.dumps(x)
3. load - Reads a pickled representation from a file and returns the reconstructed object hierarchy # pickle.load(f)
4. loads - Reads a pickled object hierarchy and returns the reconstructed object hierarchy

#### Two constants - 

v.4. pickle.HIGHEST_PROTOCOL - Supports Very Large object

v.3. pickle.DEFAULT_PROTOCOL - Supports bytes object

#### Other Classes

1. Pickler
2. Unpickler

We can pickle and unpickle anything from boolean, integer, floats, string, bytes, tuple, list, sets, dictionaries holding only pickable objects.

### unpickling

converting a byte stream into an object hierarchy

## Create Model, Migrate and Register With admin site.

***
***

## Serialize the Article Model

### 1. Using Serializer Class

The first part of the serializer class defines the fields that get serialized/deserialized. The create() and update() methods define how fully fledged instances are created or modified when calling serializer.save()

A serializer class is very similar to a Django Form class, and includes similar validation flags on the various fields, such as required, max_length and default.

The field flags can also control how the serializer should be displayed in certain circumstances, such as when rendering to HTML. The {'base_template': 'textarea.html'} flag above is equivalent to using widget=widgets.Textarea on a Django Form class. This is particularly useful for controlling how the browsable API should be displayed, as we'll see later in the tutorial.

#### View Data Using Command Line

`python manage.py shell`

`from snippets.models import Article`

`from snippets.serializers import ArticleSerializer`

`from rest_framework.renderers import JSONRenderer`

`from rest_framework.parsers import JSONParser`

`a = Article(title = 'Article Title', author="kanhaiya", email="abc@protonmail.com")`

`a = Article(title = 'Future Of Bitcoin', author="john", email="john@gmail.com")`

`a.save()`

`serializer = ArticleSerializer(a)`

`serializer.data`

`content = JSONRenderer().render(serializer.data)`  #view serialized data of single article

`content`

`serializer = ArticleSerializer(Article.objects.all(), many=True)`

`serializer.data`

`content = JSONRenderer().render(serializer.data)  #view serialized data of single article`

`content`

### 2. Model Serializers

Our ArticleSerializer class is replicating a lot of information that's also contained in the Article model. **It would be nice if we could keep our code a bit more concise**.

REST framework includes both Serializer classes, and ModelSerializer classes.

It's important to remember that ModelSerializer classes don't do anything particularly magical, they are simply a shortcut for creating serializer classes:

- An automatically determined set of fields.
- Simple default implementations for the create() and update() methods.

Let's look at refactoring our serializer using the ModelSerializer class. **Open the file Article/serializers.py again, and replace the ArticleSerializer class with the following:**

In [ ]:
#snippets/serializers.py

from rest_framework import serializers
from .models import Article

class ArticleSerializer(serializers.ModelSerializer):
    class Meta:
        model = Article
        fields = ['id','title','author']

One nice property that serializers have is that you can inspect all the fields in a serializer instance, by printing its representation. 

Open the Django shell with `python3 manage.py shell`, then try the following:

`from snippets.serializers import ArticleSerializer`

`serializer = ArticleSerializer()`

`print(repr(serializer))`

you can see it have all the elements like serializer class, but Modelserializer is a shortcut to do that.

### 3. HyperlinkedModelSerializer

The HyperlinkedModelSerializer class is similar to the ModelSerializer class except that it uses hyperlinks to represent relationships, rather than primary keys.

By default the serializer will include a url field instead of a primary key field.

The url field will be represented using a HyperlinkedIdentityField serializer field, and any relationships on the model will be represented using a HyperlinkedRelatedField serializer field.

You can explicitly include the primary key by adding it to the fields option, for example:

***
***

### Writing regular Django views using our Serializer
Let's see how we can write some API views using our new Serializer class. For the moment we won't use any of REST framework's other features, we'll just write the views as regular Django views.

Edit the snippets/views.py file, and add the following.

Note that because we want to be able to POST to this view from clients that won't have a CSRF token we need to mark the view as csrf_exempt. This isn't something that you'd normally want to do, and REST framework views actually use more sensible behavior than this, but it'll do for our purposes right now.

We'll also need a view which corresponds to an individual snippet, and can be used to retrieve, update or delete the snippet.

In [ ]:
from django.shortcuts import render
from django.http import HttpResponse, JsonResponse
from rest_framework.parsers import JSONParser
from .models import Article
from django.views.decorators.csrf import csrf_exempt
from .serializers import ArticleSerializer

@csrf_exempt
def article_list(request):
    if request.method == 'GET':
        articles = Article.objects.all()
        serializer = ArticleSerializer(articles, many=True)
        return JsonResponse(serializer.data, safe=False)

    elif request.method == 'POST':
        data = JSONParser().parse(request)
        serializer = ArticleSerializer(data=data)

        if serializer.is_valid():
            serializer.save()
            return JsonResponse(serializer.data, status=201)

        return JsonResponse(serializer.errors, status=400)

@csrf_exempt
def article_detail(request, pk):
    try:
        article = Article.objects.get(pk=pk)
    except Article.DoesNotExist:
        return HttpResponse(status=404)

    
    if request.method = 'GET':
        serializer = ArticleSerializer(article)
        return JsonResponse(serializer.data)
        
    elif request.method == 'PUT':
        data = JSONParser().parse(request)
        serializer = ArticleSerializer(article, data=data)
        if serializer.is_valid():
            serializer.save()

            return JsonResponse(serializer.data)
            return JsonResponse(serializer.errors, status=400)

    elif request.method == 'DELETE':
        article.delete()
        return HttpResponse(status=204)


***
***

***
***

# Tutorial 2

## Request objects

REST framework introduces a Request object that extends the regular HttpRequest, and provides more flexible request parsing. 

The core functionality of the Request object is the request.data attribute, which is similar to request.POST, but more useful for working with Web APIs.

In [ ]:
request.POST  # Only handles form data.  Only works for 'POST' method.
request.data  # Handles arbitrary data.  Works for 'POST', 'PUT' and 'PATCH' methods.

## Response objects
REST framework also introduces a Response object, which is a type of TemplateResponse that takes unrendered content and uses content negotiation to determine the correct content type to return to the client.

In [ ]:
return Response(data)  # Renders to content type as requested by the client.

## Status codes
Using numeric HTTP status codes in your views doesn't always make for obvious reading, and it's easy to not notice if you get an error code wrong. 

REST framework provides more explicit identifiers for each status code, such as HTTP_400_BAD_REQUEST in the status module. 

It's a good idea to use these throughout rather than using numeric identifiers.

## Wrapping API views
REST framework provides two wrappers you can use to write API views.

1. The **@api_view** decorator for working with function based views.

2. The **APIView** class for working with class-based views.

These wrappers provide a few bits of functionality such as making sure 

1. you receive Request instances in your view, 

2. and adding context to **Response(instead of httpResponse))** objects so that content negotiation can be performed. since Response renders to content type as requested by client, and in httpresponse you need to use JsonParser

The wrappers also provide behaviour such as returning 405 Method Not Allowed responses when appropriate, and handling any ParseError exceptions that occur when accessing request.data with malformed input.

1. ## api_view

- @api_view decorator for working with function based views.



In [ ]:
from django.shortcuts import render
from django.http import HttpResponse, JsonResponse
from rest_framework.parsers import JSONParser
from .models import Article
from django.views.decorators.csrf import csrf_exempt
from .serializers import ArticleSerializer
from rest_framework.decorators import api_view
from rest_framework.response import Response
from rest_framework import status

@api_view(['GET', 'POST'])
def article_list(request):
    if request.method == 'GET':
        articles = Article.objects.all()
        serializer = ArticleSerializer(articles, many=True)
        return Response(serializer.data)

    elif request.method == 'POST':
        serializer = ArticleSerializer(data=request.data)

        if serializer.is_valid():
            serializer.save()
            return Response(serializer.data, status=status.HTTP_201_CREATED)

        return Response(serializer.errors, status=status.HTTP_400_BAD_REQUEST)

@api_view(['GET','PUT','DELETE'])
def article_detail(request, pk):
    try:
        article = Article.objects.get(pk=pk)
    except Article.DoesNotExist:
        return HttpResponse(status=status.HTTP_400_BAD_REQUEST)

    
    if request.method == 'GET':
        serializer = ArticleSerializer(article)
        return Response(serializer.data)
        
    elif request.method == 'PUT':
        serializer = ArticleSerializer(article, data=request.data)
        if serializer.is_valid():
            serializer.save()

            return Response(serializer.data)
            return JsonResponse(serializer.errors, status=HTTP_400_BAD_REQUEST)

    elif request.method == 'DELETE':
        article.delete()
        return Response(status=status.HTTP_204_NO_CONTENT)

## 2. APIView class

**APIView** Class-based Views allow us to reuse common functionality, It helps us keep our code **DRY**

- Request passed to handler methods will be REST Framework's Request instance, Not Django's httpRequest instances.

In [ ]:
# snippets/views.py

from django.shortcuts import render
from django.http import HttpResponse, JsonResponse
from rest_framework.parsers import JSONParser
from .models import Article
from django.views.decorators.csrf import csrf_exempt
from .serializers import ArticleSerializer
from rest_framework.decorators import api_view
from rest_framework.views import APIView
from rest_framework.response import Response
from rest_framework import status


class ArticleAPIView(APIView):

    def get(self, request):
        articles = Article.objects.all()
        serializer = ArticleSerializer(articles, many=True)
        return Response(serializer.data)

    def post(self, request):
        serializer = ArticleSerializer(data=request.data)

        if serializer.is_valid():
            serializer.save()
            return Response(serializer.data, status=status.HTTP_201_CREATED)

        return Response(serializer.errors, status=status.HTTP_400_BAD_REQUEST)



class ArticleDetails(APIView):

    def get_object(self, id):
        try:
            return Article.objects.get(id=id)

        except Article.DoesNotExist:
            raise HttpResponse(status=status.HTTP_404_NOT_FOUND)

    def get(self, request, id):
        article = self.get_object(id)
        serializer = ArticleSerializer(article)
        return Response(serializer.data)

    def put(self, request, id):
        article = self.get_object(id)
        serializer = ArticleSerializer(article, data=request.data)
        if serializer.is_valid():
            serializer.save()

            return Response(serializer.data)
            return Response(serializer.errors, status=HTTP_400_BAD_REQUEST)


    def delete(self, request, id):
        article = self.get_object(id)
        article.delete()
        return Response(status=status.HTTP_204_NO_CONTENT)


In [ ]:
#snippets/urls.py

from django.urls import path
# from .views import article_list
# from .views import article_detail
from .views import ArticleAPIView, ArticleDetails

urlpatterns = [
    # path('article/', article_list),
    # path('detail/<int:pk>/', article_detail)
    path('article/', ArticleAPIView.as_view()),
    path('detail/<int:id>/', ArticleDetails.as_view()), #Because it's a class we need to use as views.
     
]

***
***

## 3. Generic Class-based Views

One of the key benefits of class-based views is the way they allow you to compose bits of reusable behavior. REST framework takes advantage of this by providing a number of pre-built views that provide for commonly used patterns.

The generic views provided by REST framework allow you to quickly build API views that map closely to your database models.

### Using mixins

The create/retrieve/update/delete operations that we've been using so far are going to be pretty similar for any model-backed API views we create. Those bits of common behaviour are implemented in REST framework's mixin classes.

Let's take a look at how we can compose the views by using the mixin classes. Here's our views.py module again.

The mixin classes provide the actions that are used to provide the basic view behavior. Note that the mixin classes provide action methods rather than defining the handler methods, such as .get() and .post(), directly. This allows for more flexible composition of behavior.

The ***mixin classes can be imported from rest_framework.mixins**.

1. ListModelMixin
Provides a .list(request, *args, **kwargs) method, that implements listing a queryset.

If the queryset is populated, this returns a 200 OK response, with a serialized representation of the queryset as the body of the response. The response data may optionally be paginated.

2. CreateModelMixin
Provides a .create(request, *args, **kwargs) method, that implements creating and saving a new model instance.

If an object is created this returns a 201 Created response, with a serialized representation of the object as the body of the response. If the representation contains a key named url, then the Location header of the response will be populated with that value.

If the request data provided for creating the object was invalid, a 400 Bad Request response will be returned, with the error details as the body of the response.

3. RetrieveModelMixin
Provides a .retrieve(request, *args, **kwargs) method, that implements returning an existing model instance in a response.

If an object can be retrieved this returns a 200 OK response, with a serialized representation of the object as the body of the response. Otherwise it will return a 404 Not Found.

4. UpdateModelMixin
Provides a .update(request, *args, **kwargs) method, that implements updating and saving an existing model instance.


5. Also provides a .partial_update(request, *args, **kwargs) method, which is similar to the update method, except that all fields for the update will be optional. This allows support for HTTP PATCH requests.

If an object is updated this returns a 200 OK response, with a serialized representation of the object as the body of the response.

If the request data provided for updating the object was invalid, a 400 Bad Request response will be returned, with the error details as the body of the response.

6. DestroyModelMixin
Provides a .destroy(request, *args, **kwargs) method, that implements deletion of an existing model instance.

If an object is deleted this returns a 204 No Content response, otherwise it will return a 404 Not Found.

In [ ]:
#snippets/views.py

from django.shortcuts import render
from django.http import HttpResponse, JsonResponse
from rest_framework.parsers import JSONParser
from .models import Article
from django.views.decorators.csrf import csrf_exempt
from .serializers import ArticleSerializer
from rest_framework.decorators import api_view
from rest_framework.views import APIView
from rest_framework.response import Response
from rest_framework import status

from rest_framework import generics
from rest_framework import mixins


class GenericAPIView(generics.GenericAPIView, 
                    mixins.ListModelMixin,  #get all data
                    mixins.CreateModelMixin, #post data
                    mixins.UpdateModelMixin, #put
                    mixins.RetrieveModelMixin, #get individual data
                    mixins.DestroyModelMixin): #delete
                    
    serializer_class = ArticleSerializer
    queryset = Article.objects.all()

    lookup_field = 'id'

    def get(self, request, id = None):
        if id:
            return self.retrieve(request)
        else:
            return self.list(request)

    def post(self, request):
        return self.create(request)

    def put(self, request, id=None):
        return self.update(request, id)

    def delete(self, request, id):
        return self.destroy(request, id)

In [ ]:
# snippets/urls.py

from django.urls import path
# from .views import article_list
# from .views import article_detail
# from .views import ArticleAPIView, ArticleDetails
from .views import GenericAPIView

urlpatterns = [
    # path('article/', article_list),
    # path('detail/<int:pk>/', article_detail)
    # path('article/', ArticleAPIView.as_view()),
    # path('detail/<int:id>/', ArticleDetails.as_view()), #Because it's a class we need to use as views.
    path('generic/article/<int:id>/', GenericAPIView.as_view())
     
]

****
****

The method handlers for a ViewSet are only bound to the corresponding actions at the point of finalizing the view, using the .as_view() method.

## 4. ViewSet

After routing has determined which controller to use for a request, your controller is responsible for making sense of the request and producing the appropriate output.

REST framework includes an abstraction for dealing with ViewSets, that allows the developer to concentrate on modeling the state and interactions of the API, and leave the URL construction to be handled automatically, based on common conventions.

### Routers 
Routers are used with ViewSets in django rest framework to auto config the urls. Routers provides a simple, quick and consistent way of wiring ViewSet logic to a set of URLs. Router automatically maps the incoming request to proper viewset action based on the request method type(i.e GET, POST, etc).

### i. ViewSet - 

ViewSet classes are almost the same thing as View classes, except that they provide operations such as retrieve, or update, and not method handlers such as get or put.

In [ ]:
# views.py

from django.shortcuts import render
from django.http import HttpResponse, JsonResponse
from rest_framework.parsers import JSONParser
from .models import Article
from django.views.decorators.csrf import csrf_exempt
from .serializers import ArticleSerializer
from rest_framework.decorators import api_view
from rest_framework.views import APIView
from rest_framework.response import Response
from rest_framework import status

from rest_framework import generics
from rest_framework import mixins

from rest_framework.authentication import SessionAuthentication, BasicAuthentication, TokenAuthentication
from rest_framework.permissions import IsAuthenticated
from django.shortcuts import get_list_or_404, get_object_or_404


from rest_framework import viewsets

class ArticleViewSet(viewsets.ViewSet):

    def list(self, request):
        articles = Article.objects.all()
        serializer = ArticleSerializer(articles, many=True)
        return Response(serializer.data)

    def create(self, request): #post
        serializer = ArticleSerializer(data=request.data)

        if serializer.is_valid():
            serializer.save()
            return Response(serializer.data, status=status.HTTP_201_CREATED)

        return Response(serializer.errors, status=status.HTTP_400_BAD_REQUEST)

    def retrieve(self, request, pk=None): #get
        queryset = Article.objects.all()
        article = get_object_or_404(queryset, pk=pk)
        serializer = ArticleSerializer(article)
        return Response(serializer.data)

    def update(self, request, pk=None): #put
        article = Article.objects.get(pk=pk)
        serializer = ArticleSerializer(article, data=request.data)
        if serializer.is_valid():
            serializer.save()

            return Response(serializer.data)
            return Response(serializer.errors, status=HTTP_400_BAD_REQUEST)

In [ ]:
#urls.py

from django.urls import path, include
# from .views import article_list
# from .views import article_detail
# from .views import ArticleAPIView, ArticleDetails
from .views import GenericAPIView, ArticleViewSet
from rest_framework.routers import DefaultRouter

router = DefaultRouter()
router.register('article', ArticleViewSet, basename='article')

urlpatterns = [
    path('viewset/', include(router.urls)),
    path('viewset/<int:pk>/', include(router.urls)),
]

### ii. GenericViewSet - 

The GenericViewSet class inherits from GenericAPIView, and provides the default set of get_object, get_queryset methods and other generic view base behavior, but does not include any actions by default.

In order to use a GenericViewSet class you'll override the class and either mixin the required mixin classes, or define the action implementations explicitly.

In [ ]:
#example

class ArticleViewSet(viewsets.GenericViewSet, mixins.ListModelMixin):
    serializer_class = ArticleSerializer
    queryset = Article.objects.all()
    
#now you don't need to add get() method to get the data since it is GenericViewSet..prebuild abstracted view
#autoimplemented get method using ListModelMixin

### iii. ModelViewSet

The ModelViewSet class inherits from GenericAPIView and includes implementations for various actions, by mixing in the behavior of the various mixin classes.

The actions provided by the ModelViewSet class are .list(), .retrieve(), .create(), .update(), .partial_update(), and .destroy().

In [ ]:
class ArticleViewSet(viewsets.ModelViewSet):
    serializer_class = ArticleSerializer
    queryset = Article.objects.all()

In [ ]:
# urls.py

from django.urls import path, include
# from .views import article_list
# from .views import article_detail
# from .views import ArticleAPIView, ArticleDetails
from .views import GenericAPIView, ArticleViewSet
from rest_framework.routers import DefaultRouter

router = DefaultRouter()
router.register('article', ArticleViewSet, basename='article')

urlpatterns = [
    path('viewset/', include(router.urls)),
    path('viewset/<int:pk>/', include(router.urls)),
    # path('article/', article_list),
    # path('detail/<int:pk>/', article_detail)
    # path('article/', ArticleAPIView.as_view()),
    # path('detail/<int:id>/', ArticleDetails.as_view()), #Because it's a class we need to use as views.
    path('generic/article/<int:id>/', GenericAPIView.as_view())
     
]

### iv. ReadOnlyModelViewSet

ReadOnlyModelViewSet class also inherits from GenericAPIView. As with ModelViewSet it also includes implementations for various actions, but unlike ModelViewSet only provides the 'read-only' actions, .list() and .retrieve().

In [ ]:
# As with ModelViewSet, you'll normally need to provide at least the queryset and serializer_class attributes. For example:

class AccountViewSet(viewsets.ReadOnlyModelViewSet):
    """
    A simple ViewSet for viewing accounts.
    """
    queryset = Account.objects.all()
    serializer_class = AccountSerializer

### v. Custom ViewSet base classes
You may need to provide custom ViewSet classes that do not have the full set of ModelViewSet actions, or that customize the behavior in some other way.

Example
To create a base viewset class that provides create, list and retrieve operations, inherit from GenericViewSet, and mixin the required actions:


In [ ]:
from rest_framework import mixins

class CreateListRetrieveViewSet(mixins.CreateModelMixin,
                                mixins.ListModelMixin,
                                mixins.RetrieveModelMixin,
                                viewsets.GenericViewSet):
    """
    A viewset that provides `retrieve`, `create`, and `list` actions.

    To use it, override the class and set the `.queryset` and
    `.serializer_class` attributes.
    """
    pass

# Hyperlinking our API
Dealing with relationships between entities is one of the more challenging aspects of Web API design. There are a number of different ways that we might choose to represent a relationship:

- Using primary keys.
- Using hyperlinking between entities.
- Using a unique identifying slug field on the related entity.
- Using the default string representation of the related entity.
- Nesting the related entity inside the parent representation.
- Some other custom representation.
- REST framework supports all of these styles, and can apply them across forward or reverse relationships, or apply them across custom managers such as generic foreign keys.

### HyperlinkedModelSerializer 

has the following differences from ModelSerializer:

It does not include the id field by default.

It includes a url field, using HyperlinkedIdentityField.

Relationships use HyperlinkedRelatedField, instead of PrimaryKeyRelatedField.

## Adding optional format suffixes to our URLs

To take advantage of the fact that our responses are no longer hardwired to a single content type let's add support for format suffixes to our API endpoints. Using format suffixes gives us URLs that explicitly refer to a given format, and means our API will be able to handle URLs such as http://example.com/api/items/4.json.

Start by adding a format keyword argument to both of the views, like so.

def snippet_list(request, format=None):

&

def snippet_detail(request, pk, format=None):

In [ ]:
#Now update the snippets/urls.py file slightly, to append a set of format_suffix_patterns..
#...in addition to the existing URLs.

from django.urls import path
from rest_framework.urlpatterns import format_suffix_patterns
from snippets import views

urlpatterns = [
    path('snippets/', views.snippet_list),
    path('snippets/<int:pk>', views.snippet_detail),
]

urlpatterns = format_suffix_patterns(urlpatterns)

***
***

# API endpoint

URLs with a list of available actions (HTTP verbs) that expose data

An endpoint is one end of a communication channel, Where an API interacts with another system.

## Status Codes

Once your web browser has executed an HTTP Request on a URL there is no guarantee things will actually work! Thus there is a quite lengthy list of HTTP Status Codes available to accompany each HTTP response.

You can tell the general type of status code based on the following system:

- 2xx Success - the action requested by the client was received, understood, and accepted
- 3xx Redirection - the requested URL has moved
- 4xx Client Error - there was an error, typically a bad URL request by the client
- 5xx Server Error - the server failed to resolve a request

here is no need to memorize all the available status codes. With practice you will become familiar with the most common ones such as 200 (OK), 201 (Created), 301 (Moved Permanently), 404 (Not Found), and 500 (Server Error).

The important thing to remember is that, generally speaking, there are only four potential outcomes to any given HTTP request: it worked (2xx), it was redirected somehow (3xx), the client made an error (4xx), or the server made an error (5xx).

## Statelessness

A final important point to make about HTTP is that it is a stateless protocol. This means each request/response pair is completely independent of the previous one. There is no stored memory of past interactions, which is known as state in computer science.

### Difference in get and filter:

- use get() if you know there is only single object that matches your query.
- use filter() when you want to get all objects that match your lookup parameters.

mymodel=model.objects.get(name='pol')

mymodel=model.objects.filter(name='pol')

***
***